In [1]:
# Add parent folder to path
import sys
sys.path.append("..")

import numpy as np
import scipy.spatial

# Own stuff
from src.Kernels.Gaussian import Gaussian
from src.Particle import Particle
from src.Equations.WCSPH import WCSPH
from src.Animation2 import Animation
from src.Integrators.EulerIntegrater import EulerIntegrater

# tqdm
from tqdm.autonotebook import tqdm

C:\iPython3\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
def create_particles(wcsph, N: int, mass: float):
    # Create some particles
    xv = np.linspace(0, 2, N)
    yv = np.linspace(0, 2, N)
    x, y = np.meshgrid(xv, yv, indexing='ij')
    x = x.ravel()
    y = y.ravel()

    particles = []
    for i in range(len(x)):
        particles.append(Particle('fluid', x[i], y[i], mass))
    
    r0 = 1 / N # Distance between boundary particles.
    rho_b = 1008. # Density of the boundary [kg/m^3]
    mass_b = mass * 2 # Mass of the boundary [kg]

    # Maximum and minimum values of boundaries
    x_min = -0.5
    x_max = 10.1
    y_min = -0.5
    y_max = 5
    
    # Bottom & Top
    xv = np.arange(x_min, x_max, r0)
    yv = np.zeros(len(xv)) + y_min
    #yv2 = np.zeros(len(xv)) + 10
    for i in range(len(xv)):
        particles.append(Particle('boundary', xv[i], yv[i], mass_b, rho_b))
        particles.append(Particle('boundary', xv[i] - r0 / 2, yv[i] - r0, mass_b, rho_b))
        particles.append(Particle('boundary', xv[i], yv[i] - 2 * r0, mass_b, rho_b))
        particles.append(Particle('boundary', xv[i] - r0 / 2, yv[i] - 3 * r0, mass_b, rho_b))
        #particles.append(Particle('boundary', xv[i], yv2[i] + r0, rho_b))

    # Left & Right
    yv3 = np.arange(y_min, y_max, r0)    
    xv2 = np.zeros(len(yv3)) + x_min
    xv3 = np.zeros(len(yv3)) + x_max
    for i in range(len(yv3)):
        particles.append(Particle('boundary', xv2[i], yv3[i], mass_b, rho_b))
        particles.append(Particle('boundary', xv2[i] - r0, yv3[i] - r0 / 2, mass_b, rho_b))
        particles.append(Particle('boundary', xv3[i], yv3[i], mass_b, rho_b))
        particles.append(Particle('boundary', xv3[i] + r0, yv3[i] - r0 / 2, mass_b, rho_b))
        
    return particles

## Calculation

In [3]:
def calc_smoothing_length(mass, rho_a: np.array) -> np.array:
    return 1.0 * np.sqrt(mass / rho_a)

In [4]:
# WCSPH method
wcsph = WCSPH(height=2.0)

# Generate some particles
N = 10
mass = 2 * 2 * wcsph.rho0 / (N ** 2) # Per particle
particles = create_particles(wcsph, N, mass)

fluid_particles = [p for p in particles if p.label == 'fluid']

# Get fluid indices
fluid_ind = []
for i in range(len(particles)):
    if particles[i].label == 'fluid':
        fluid_ind.append(i)
fluid_ind = np.array(fluid_ind)

# Solving properties
kernel = Gaussian()
integrater = EulerIntegrater()
t_max = 1.5
dt = 0.02
t = np.arange(0, t_max, dt)
t_n = len(t)
H = 2 # water column height
hdx = 1.3 * np.sqrt(H * 2 * 2 / len(fluid_particles))
h = np.ones(len(particles)) * hdx

gx = 0.
gy = -9.81

# Initialize the loop
for p in particles:
    wcsph.inital_condition(p)

# Time-stepping
x = np.zeros([len(particles), t_n]) # X-pos
y = np.zeros([len(particles), t_n]) # Y-pos
c = np.zeros([len(particles), t_n]) # Pressure (p)
u = np.zeros([len(particles), t_n]) # Density (rho)
v = np.zeros([len(particles), 2, t_n]) # Velocity both x and y


# Initialization
x[:, 0] = [p.r[0] for p in particles]
y[:, 0] = [p.r[1] for p in particles]
c[:, 0] = [p.p for p in particles]
u[:, 0] = [p.rho for p in particles]

i: int = 0
for t_step in tqdm(range(t_n - 1), desc='Time-stepping'):
    # Distance and neighbourhood
    r = np.array([p.r for p in particles])
    dist = scipy.spatial.distance.cdist(r, r, 'euclidean')
    hood = scipy.spatial.cKDTree(r)
    
    # Force/Acceleration evaluation loop
    i: int = 0
    for p in tqdm(particles, desc='Evaluating equations', leave=False):
        if p.label == 'boundary':
            continue
        
        wcsph.loop_initialize(p)
        
        # Query neighbours
        r_dist: float  = 3 * h[0] # Goes to zero when q > 3
        near_ind: list = hood.query_ball_point(p.r, r_dist)
        near_arr: np.array = np.array(near_ind)
        
        # Skip if got no neighbours
        # Keep same properties, no acceleration.
        if len(near_arr) == 0:
            continue
        
        # Calculate some general properties
        xij: np.array = r[near_arr] - p.r
        rij: np.array = dist[i, near_arr]
        vij: np.array = v[near_arr, :, t_step] - p.v
        dwij: np.array = kernel.gradient(xij, rij, h[near_arr])
        
        # Evaluate the equations
        wcsph.TaitEOS(p)
        wcsph.Continuity(mass, p, xij, rij, dwij, vij)
        wcsph.Momentum(mass, p, c[near_arr, t_step], u[near_arr, t_step], dwij)
        wcsph.Gravity(p, gx, gy)
        
        i += 1
    
    # Integration loop
    i: int = 0
    for p in particles:
        # Integrate the thingies
        integrater.integrate(dt, p)
        
        # Put into giant-matrix
        x[i, t_step + 1] = p.r[0]
        y[i, t_step + 1] = p.r[1]
        c[i, t_step + 1] = p.p
        u[i, t_step + 1] = p.rho
        v[i, :, t_step + 1] = p.v
        
        i += 1
        
# Export
fluid_ind = []
solid_ind = []
i: int = 0
for p in particles:
    if p.label == 'fluid':
        fluid_ind.append(i)
    else:
        solid_ind.append(i)
    i += 1

ani = Animation(
    x=x[fluid_ind], 
    y=y[fluid_ind], 
    r=3.0, 
    c=c[fluid_ind], 
    fps=10, 
    xlim=[-1, 11], 
    ylim=[-1, 11], 
    xsolid=x[solid_ind], 
    ysolid=y[solid_ind])
ani.export('hope4.mp4', codec='libx265')

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

...

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

TypeError: 'float' object cannot be interpreted as an integer

In [5]:
import numpy as np
import pyqtgraph as pg
import pyqtgraph.exporters

In [102]:
# Construct scatter and initial frame
r = []
frame = 0
c_max = max(c[:, frame])
color = []
for i in range(len(x)):
    r.append({'x': x[i, frame], 'y': y[i, frame]})
    nn = int(round((c[i, frame] / c_max) * 255, 0))
    color.append(pg.mkPen(nn, nn, nn, 1.0))
plt = pg.plot(r, pen=None, symbolPen=color, symbol='o', symbolSize=2)
plt.setXRange(-1, 11)
plt.setYRange(-1, 5)
exporter = pg.exporters.ImageExporter(plt.plotItem)

exporter.params.param('width').setValue(700, blockSignal=exporter.widthChanged)
exporter.params.param('height').setValue(500, blockSignal=exporter.heightChanged)

fileName = f'export_{frame}.png'
exporter.export(fileName)

In [94]:
color[0].color

<function QBrush.color>

In [13]:
pg.exporters.ImageExporter(sctr).export('test.png')

TypeError: 'float' object cannot be interpreted as an integer

In [95]:
# generate something to export
plt = pg.plot([1,5,2,4,3])

pg.setConfigOption('background', 'w')
pg.setConfigOption('foreground', 'k')

# create an exporter instance, as an argument give it
# the item you wish to export
exporter = pg.exporters.ImageExporter(plt.plotItem)

# set export parameters if needed
exporter.params.param('width').setValue(1920, blockSignal=exporter.widthChanged)
exporter.params.param('height').setValue(1080, blockSignal=exporter.heightChanged)

# save to file
exporter.export('fileName.png')